# 如何在ReAct智能体中管理对话历史

!!! info "先决条件"
    本指南假设您熟悉以下内容：

    - [预构建的create_react_agent](../create-react-agent)
    - [持久化](../../concepts/persistence)
    - [短期内存](../../concepts/memory/#short-term-memory)
    - [消息修剪](https://python.langchain.com/docs/how_to/trim_messages/)

消息历史可能会快速增长并超过LLM上下文窗口大小，无论您是在构建具有多次对话轮次的聊天机器人，还是构建具有大量工具调用的智能体系统。有几种管理消息历史的策略：

* [消息修剪](#keep-the-original-message-history-unmodified) — 删除历史记录中的前N条或后N条消息
* [摘要](#summarizing-message-history) — 总结历史记录中的早期消息并用摘要替换它们
* 自定义策略（例如，消息过滤等）

要在`create_react_agent`中管理消息历史，您需要定义一个`pre_model_hook`函数或[可运行对象](https://python.langchain.com/docs/concepts/runnables/)，该函数接受图状态并返回状态更新：


* 修剪示例：
    ```python
    # highlight-next-line
    from langchain_core.messages.utils import (
        # highlight-next-line
        trim_messages, 
        # highlight-next-line
        count_tokens_approximately
    # highlight-next-line
    )
    from langgraph.prebuilt import create_react_agent
    
    # 此函数将在调用LLM的节点之前每次被调用
    def pre_model_hook(state):
        trimmed_messages = trim_messages(
            state["messages"],
            strategy="last",
            token_counter=count_tokens_approximately,
            max_tokens=384,
            start_on="human",
            end_on=("human", "tool"),
        )
        # 您可以在`llm_input_messages`或`messages`键下返回更新的消息
        # （请参见下面的注释）
        # highlight-next-line
        return {"llm_input_messages": trimmed_messages}

    checkpointer = InMemorySaver()
    agent = create_react_agent(
        model,
        tools,
        # highlight-next-line
        pre_model_hook=pre_model_hook,
        checkpointer=checkpointer,
    )
    ```

* 摘要示例：
    ```python
    # highlight-next-line
    from langmem.short_term import SummarizationNode
    from langchain_core.messages.utils import count_tokens_approximately
    from langgraph.prebuilt.chat_agent_executor import AgentState
    from langgraph.checkpoint.memory import InMemorySaver
    from typing import Any
    
    model = ChatOpenAI(model="gpt-4o")
    
    summarization_node = SummarizationNode(
        token_counter=count_tokens_approximately,
        model=model,
        max_tokens=384,
        max_summary_tokens=128,
        output_messages_key="llm_input_messages",
    )

    class State(AgentState):
        # 注意：我们添加此键来跟踪先前的摘要信息
        # 以确保我们不会在每次LLM调用时都进行摘要
        # highlight-next-line
        context: dict[str, Any]
    
    
    checkpointer = InMemorySaver()
    graph = create_react_agent(
        model,
        tools,
        # highlight-next-line
        pre_model_hook=summarization_node,
        # highlight-next-line
        state_schema=State,
        checkpointer=checkpointer,
    )
    ```

!!! Important
    
    * 要在图状态中**保持原始消息历史不变**并**仅将更新的历史作为LLM的输入**传递，请在`llm_input_messages`键下返回更新的消息
    * 要用更新的历史**覆盖图状态中的原始消息历史**，请在`messages`键下返回更新的消息
    
    要覆盖`messages`键，您需要执行以下操作：

    ```python
    from langchain_core.messages import RemoveMessage
    from langgraph.graph.message import REMOVE_ALL_MESSAGES

    def pre_model_hook(state):
        updated_messages = ...
        return {
            "messages": [RemoveMessage(id=REMOVE_ALL_MESSAGES), *updated_messages]
            ...
        }
    ```


## 设置

首先，让我们安装所需的包并设置API密钥


In [ ]:
%%capture --no-stderr
%pip install -U langgraph langchain-openai langmem


In [ ]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")


<div class="admonition tip">
    <p class="admonition-title">为LangGraph开发设置 <a href="https://smith.langchain.com">LangSmith</a></p>
    <p style="padding-top: 5px;">
        注册LangSmith以快速发现问题并提高LangGraph项目的性能。LangSmith让您使用跟踪数据来调试、测试和监控使用LangGraph构建的LLM应用程序——阅读更多关于如何开始的信息 <a href="https://docs.smith.langchain.com">这里</a>。 
    </p>
</div>


## 保持原始消息历史不变


让我们构建一个带有管理对话历史步骤的ReAct智能体：当历史长度超过指定的令牌数量时，我们将调用[`trim_messages`](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.utils.trim_messages.html)实用程序，该程序将减少历史记录，同时满足LLM提供商的约束。

在ReAct智能体内部应用更新的消息历史有两种方式：

  * [**保持原始消息历史不变**](#keep-the-original-message-history-unmodified)在图状态中，并**仅将更新的历史作为LLM的输入**传递
  * [**用更新的历史覆盖原始消息历史**](#overwrite-the-original-message-history)在图状态中

让我们从实现第一个开始。我们首先需要为我们的智能体定义模型和工具：


In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", temperature=0)


def get_weather(location: str) -> str:
    """使用此工具获取天气信息。"""
    if any([city in location.lower() for city in ["nyc", "new york city"]]):
        return "纽约市可能多云，有下雨的可能，温度高达80度。"
    elif any([city in location.lower() for city in ["sf", "san francisco"]]):
        return "旧金山总是阳光明媚"
    else:
        return f"我不确定{location}的天气情况"


tools = [get_weather]


现在让我们实现`pre_model_hook`——一个将作为新节点添加并在调用LLM的节点（`agent`节点）**之前**每次被调用的函数。

我们的实现将包装`trim_messages`调用并在`llm_input_messages`下返回修剪后的消息。这将**在图状态中保持原始消息历史不变**并**仅将更新的历史作为LLM的输入**传递


In [ ]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver

# highlight-next-line
from langchain_core.messages.utils import (
    # highlight-next-line
    trim_messages,
    # highlight-next-line
    count_tokens_approximately,
    # highlight-next-line
)


# 此函数将作为ReAct智能体图中的新节点添加
# 在调用LLM的节点之前每次运行。
# 此函数返回的消息将是LLM的输入。
def pre_model_hook(state):
    trimmed_messages = trim_messages(
        state["messages"],
        strategy="last",
        token_counter=count_tokens_approximately,
        max_tokens=384,
        start_on="human",
        end_on=("human", "tool"),
    )
    # highlight-next-line
    return {"llm_input_messages": trimmed_messages}


checkpointer = InMemorySaver()
graph = create_react_agent(
    model,
    tools,
    # highlight-next-line
    pre_model_hook=pre_model_hook,
    checkpointer=checkpointer,
)
